<a href="https://colab.research.google.com/github/Daimonic7/CEE490/blob/main/HW_5_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#PART A
import numpy as np
import sympy as sp
import sys
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

sp.init_printing(use_latex=True)

c = 16.35 # g/l
h = 0.15
x = sp.Function('x')
t = sp.symbols('t')

sol = sp.dsolve(x(t).diff(t,2) + c*x(t), ics={x(0): 10, x(t).diff(t).subs(t,0): 0})
print(sol)

x_val = np.arange(0,6,h)
plt.title("Exact Solution")
plt.plot(x_val, [sol.subs(t, i).rhs for i in x_val])
plt.xlabel("time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()

def EEulerMethod(fp1, fp2, x10, x20, h, t0, tmax):
  n = int((tmax - t0)/h + 1)
  x1table = [0 for i in range(n)]
  x2table = [0 for i in range(n)]
  x1table[0] = x10
  x2table[0] = x20
  for i in range(1,n):
    x1table[i] = x1table[i - 1] +  h*fp1(x1table[i - 1], x2table[i - 1], t0 + (i - 1)*h)
    x2table[i] = x2table[i - 1] +  h*fp2(x1table[i - 1], x2table[i - 1], t0 + (i - 1)*h) 
  Data = [[t0 + i*h, x1table[i], x2table[i]] for i in range(n)]
  return Data

def IEulerMethod(fp1, fp2, x10, x20, h, t0, tmax):
  x1 = sp.symbols('x1')
  x2 = sp.symbols('x2')
  n = int((tmax - t0)/h + 1)
  x1table = [0 for i in range(n)]
  x2table = [0 for i in range(n)]
  x1table[0] = x10
  x2table[0] = x20
  for i in range(1,n):
    #s = sp.nsolve(x - xtable[i - 1] - h*fp(x, t0 + i*h), x, xtable[i - 1])  
    eq1 = x1 - x1table[i - 1] - h*fp1(x1, x2, t0 + (i)*h)
    eq2 = x2 - x2table[i - 1] - h*fp2(x1, x2, t0 + (i)*h)
    s1, s2 = sp.nsolve((eq1, eq2), (x1, x2), (x1table[i - 1], x2table[i - 1]))
    x1table[i] = s1
    x2table[i] = s2
  Data = [[t0 + i*h, x1table[i], x2table[i]] for i in range(n)]
  return Data

def RK2Method(fp1, fp2, x10, x20, h, t0, tmax):
  n = int((tmax - t0)/h + 1)
  x1table = [0 for i in range(n)]
  x1table[0] = x10
  x2table = [0 for i in range(n)]
  x2table[0] = x20
  alpha = 0.5
  for i in range(1,n):
        k11 = fp1(x1table[i - 1], x2table[i - 1], t0 + (i - 1)*h)
        k12 = fp1(x1table[i - 1] + alpha*h, x2table[i - 1] + alpha*k11, t0 + (i - 1)*h)
        k21 = fp2(x1table[i - 1], x2table[i - 1], t0 + (i - 1)*h)
        k22 = fp2(x1table[i - 1] + alpha*h, x2table[i - 1] + alpha*k21, t0 + (i - 1)*h)

        x1table[i] = x1table[i - 1] + h*(k11 + 2*k12)/6
        x2table[i] = x2table[i - 1] + h*(k21 + 2*k22)/6
        Data = [[t0 + i*h, x1table[i], x2table[i]] for i in range(n)]
  return Data

def RK4Method(fp1, fp2, x10, x20, h, t0, tmax):
  n = int((tmax - t0)/h + 1)
  x1table = [0 for i in range(n)]
  x1table[0] = x10
  x2table = [0 for i in range(n)]
  x2table[0] = x20
  for i in range(1,n):
    k11 = fp1(x1table[i - 1], x2table[i - 1], t0 + (i - 1)*h)
    k21 = fp2(x1table[i - 1], x2table[i - 1], t0 + (i - 1)*h)
    k12 = fp1(x1table[i - 1] + h/2*k11, x2table[i - 1] + h/2*k21, t0 + (i - 0.5)*h)
    k22 = fp2(x1table[i - 1] + h/2*k11, x2table[i - 1] + h/2*k21, t0 + (i - 0.5)*h)
    k13 = fp1(x1table[i - 1] + h/2*k12, x2table[i - 1] + h/2*k22, t0 + (i - 0.5)*h)
    k23 = fp2(x1table[i - 1] + h/2*k12, x2table[i - 1] + h/2*k22, t0 + (i - 0.5)*h)
    k14 = fp1(x1table[i - 1] + h*k13, x2table[i - 1] + h*k23, t0 + i*h)
    k24 = fp2(x1table[i - 1] + h*k13, x2table[i - 1] + h*k23, t0 + i*h)
    x1table[i] = x1table[i - 1] + h*(k11 + 2*k12 + 2*k13 + k14)/6
    x2table[i] = x2table[i - 1] + h*(k21 + 2*k22 + 2*k23 + k24)/6
  Data = [[t0 + i*h, x1table[i], x2table[i]] for i in range(n)]
  return Data

def trapezoidal(fp1, fp2, x10, x20, h, t0, tmax):
  x1 = sp.symbols('x1')
  x2 = sp.symbols('x2')
  n = int((tmax - t0)/h + 1)
  x1table = [0 for i in range(n)]
  x2table = [0 for i in range(n)]
  x1table[0] = x10
  x2table[0] = x20
  for i in range(1,n): 
    eq1 = x1 - x1table[i - 1] - (h/2)*(fp1(x1table[i - 1], x2table[i - 1], t0 + (i)*h) + fp1(x1, x2, t0 + (i)*h))
    eq2 = x2 - x2table[i - 1] - (h/2)*(fp2(x1table[i - 1], x2table[i - 1], t0 + (i)*h) + fp2(x1, x2, t0 + (i)*h))
    s1, s2 = sp.nsolve((eq1, eq2), (x1, x2), (x1table[i - 1], x2table[i - 1]))
    x1table[i] = s1
    x2table[i] = s2
  Data = [[t0 + i*h, x1table[i], x2table[i]] for i in range(n)]
  return Data

def fp1(x, y, t): return y
def fp2(x, y, t): return -c*x 

Data = EEulerMethod(fp1, fp2, 10, 0, h, 0, 6)
plt.title("Explicit Euler")
plt.plot(x_val, [sol.subs(t, i).rhs for i in x_val])
plt.scatter([DataPosition[0] for DataPosition in Data],
            [DataPosition[1] for DataPosition in Data],c='k')
plt.xlabel("Time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()

Data = IEulerMethod(fp1, fp2, 10, 0, h, 0, 6)
plt.title("Implicit Euler")
plt.plot(x_val, [sol.subs(t, i).rhs for i in x_val])
plt.scatter([DataPosition[0] for DataPosition in Data],
            [DataPosition[1] for DataPosition in Data],c='k')
plt.xlabel("Time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()

Data = RK2Method(fp1, fp2, 10, 0, h, 0, 6)
plt.title("2nd Order Runge-Kutta")
plt.plot(x_val, [sol.subs(t, i).rhs for i in x_val])
plt.scatter([DataPosition[0] for DataPosition in Data],
            [DataPosition[1] for DataPosition in Data],c='k')
plt.xlabel("Time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()

Data = RK4Method(fp1, fp2, 10, 0, h, 0, 6)
plt.title("Classical Runge-Kutta")
plt.plot(x_val, [sol.subs(t, i).rhs for i in x_val])
plt.scatter([DataPosition[0] for DataPosition in Data],
            [DataPosition[1] for DataPosition in Data],c='k')
plt.xlabel("Time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()

Data = trapezoidal(fp1, fp2, 10, 0, h, 0, 6)
plt.title("Trapezoidal")
plt.plot(x_val, [sol.subs(t, i).rhs for i in x_val])
plt.scatter([DataPosition[0] for DataPosition in Data],
            [DataPosition[1] for DataPosition in Data],c='k')
plt.xlabel("Time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()

In [ ]:
#PART B
import numpy as np
import sympy as sp
import sys
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

sp.init_printing(use_latex=True)

gol = 16.35 # g/l
c = 4
h = 0.15
x = sp.Function('x')
t = sp.symbols('t')

sol = sp.dsolve(x(t).diff(t,2) + gol*x(t) + c*x(t).diff(t), ics={x(0): 10, x(t).diff(t).subs(t,0): 0})
print(sol)

x_val = np.arange(0,6,h)
plt.title("Exact Solution")
plt.plot(x_val, [sol.subs(t, i).rhs for i in x_val])
plt.xlabel("time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()

def EEulerMethod(fp1, fp2, x10, x20, h, t0, tmax):
  n = int((tmax - t0)/h + 1)
  x1table = [0 for i in range(n)]
  x2table = [0 for i in range(n)]
  x1table[0] = x10
  x2table[0] = x20
  for i in range(1,n):
    x1table[i] = x1table[i - 1] +  h*fp1(x1table[i - 1], x2table[i - 1], t0 + (i - 1)*h)
    x2table[i] = x2table[i - 1] +  h*fp2(x1table[i - 1], x2table[i - 1], t0 + (i - 1)*h) 
  Data = [[t0 + i*h, x1table[i], x2table[i]] for i in range(n)]
  return Data

def IEulerMethod(fp1, fp2, x10, x20, h, t0, tmax):
  x1 = sp.symbols('x1')
  x2 = sp.symbols('x2')
  n = int((tmax - t0)/h + 1)
  x1table = [0 for i in range(n)]
  x2table = [0 for i in range(n)]
  x1table[0] = x10
  x2table[0] = x20
  for i in range(1,n):
    #s = sp.nsolve(x - xtable[i - 1] - h*fp(x, t0 + i*h), x, xtable[i - 1])  
    eq1 = x1 - x1table[i - 1] - h*fp1(x1, x2, t0 + (i)*h)
    eq2 = x2 - x2table[i - 1] - h*fp2(x1, x2, t0 + (i)*h)
    s1, s2 = sp.nsolve((eq1, eq2), (x1, x2), (x1table[i - 1], x2table[i - 1]))
    x1table[i] = s1
    x2table[i] = s2
  Data = [[t0 + i*h, x1table[i], x2table[i]] for i in range(n)]
  return Data

def RK2Method(fp1, fp2, x10, x20, h, t0, tmax):
  n = int((tmax - t0)/h + 1)
  x1table = [0 for i in range(n)]
  x1table[0] = x10
  x2table = [0 for i in range(n)]
  x2table[0] = x20
  alpha = 0.5
  for i in range(1,n):
        k11 = fp1(x1table[i - 1], x2table[i - 1], t0 + (i - 1)*h)
        k12 = fp1(x1table[i - 1] + alpha*h, x2table[i - 1] + alpha*k11, t0 + (i - 1)*h)
        k21 = fp2(x1table[i - 1], x2table[i - 1], t0 + (i - 1)*h)
        k22 = fp2(x1table[i - 1] + alpha*h, x2table[i - 1] + alpha*k21, t0 + (i - 1)*h)

        x1table[i] = x1table[i - 1] + h*(k11 + 2*k12)/6
        x2table[i] = x2table[i - 1] + h*(k21 + 2*k22)/6
        Data = [[t0 + i*h, x1table[i], x2table[i]] for i in range(n)]
  return Data

def fp1(x, y, t): return y
def fp2(x, y, t): return -c*y - gol*x 

Data = EEulerMethod(fp1, fp2, 10, 0, h, 0, 6)
plt.title("Explicit Euler")
plt.plot(x_val, [sol.subs(t, i).rhs for i in x_val])
plt.scatter([DataPosition[0] for DataPosition in Data],
            [DataPosition[1] for DataPosition in Data],c='k')
plt.xlabel("Time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()

Data = IEulerMethod(fp1, fp2, 10, 0, h, 0, 6)
plt.title("Implicit Euler")
plt.plot(x_val, [sol.subs(t, i).rhs for i in x_val])
plt.scatter([DataPosition[0] for DataPosition in Data],
            [DataPosition[1] for DataPosition in Data],c='k')
plt.xlabel("Time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()

Data = RK2Method(fp1, fp2, 10, 0, h, 0, 6)
plt.title("2nd Order Runge-Kutta")
plt.plot(x_val, [sol.subs(t, i).rhs for i in x_val])
plt.scatter([DataPosition[0] for DataPosition in Data],
            [DataPosition[1] for DataPosition in Data],c='k')
plt.xlabel("Time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()

In [ ]:
# PART C
import numpy as np
import sympy as sp
import sys
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

sp.init_printing(use_latex=True)

c = 16.35 # g/l
h = 0.15
x = sp.Function('x')
t = sp.symbols('t')

def ltrapezoidal(fp1, fp2, x10, x20, h, t0, tmax):
  x1 = sp.symbols('x1')
  x2 = sp.symbols('x2')
  n = int((tmax - t0)/h + 1)
  x1table = [0 for i in range(n)]
  x2table = [0 for i in range(n)]
  x1table[0] = x10
  x2table[0] = x20
  for i in range(1,n): 
    eq1 = x1 - x1table[i - 1] - (h/2)*(fp1(x1table[i - 1], x2table[i - 1], t0 + (i)*h) + fp1(x1, x2, t0 + (i)*h))
    eq2 = x2 - x2table[i - 1] - (h/2)*(fp2(x1table[i - 1], x2table[i - 1], t0 + (i)*h) + fp2(x1, x2, t0 + (i)*h))
    s1, s2 = sp.nsolve((eq1, eq2), (x1, x2), (x1table[i - 1], x2table[i - 1]))
    x1table[i] = s1
    x2table[i] = s2
  Data = [[t0 + i*h, x1table[i], x2table[i]] for i in range(n)]
  return Data

def nltrapezoidal(fp1, fp2, x10, x20, h, t0, tmax):
  x1 = sp.symbols('x1')
  x2 = sp.symbols('x2')
  n = int((tmax - t0)/h + 1)
  x1table = [0 for i in range(n)]
  x2table = [0 for i in range(n)]
  x1table[0] = x10
  x2table[0] = x20
  for i in range(1,n): 
    eq1 = x1 - x1table[i - 1] - (h/2)*(fp1(x1table[i - 1], x2table[i - 1], t0 + (i)*h) + fp1(x1, x2, t0 + (i)*h))
    eq2 = x2 - x2table[i - 1] - (h/2)*(fp2nl(x1table[i - 1], x2table[i - 1], t0 + (i)*h) + fp2nl(x1, x2, t0 + (i)*h))
    s1, s2 = sp.nsolve((eq1, eq2), (x1, x2), (x1table[i - 1], x2table[i - 1]))
    x1table[i] = s1
    x2table[i] = s2
  Data = [[t0 + i*h, x1table[i], x2table[i]] for i in range(n)]
  return Data


def fp1(x, y, t): return y
def fp2(x, y, t): return -c*x 
def fp2nl(x, y, t): return -c*sp.sin(x) 

x_val = np.arange(0,6,h)

Data = ltrapezoidal(fp1, fp2, 10, 0, h, 0, 6)
plt.title("Trapezoidal")
plt.scatter([DataPosition[0] for DataPosition in Data],
            [DataPosition[1] for DataPosition in Data],c='k')
plt.xlabel("Time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()

Data = nltrapezoidal(fp1, fp2nl, 10, 0, h, 0, 6)
plt.title("NLTrapezoidal")
plt.scatter([DataPosition[0] for DataPosition in Data],
            [DataPosition[1] for DataPosition in Data],c='k')
plt.xlabel("Time"); plt.ylabel("x (angle)")
plt.grid(); plt.show()